In [1]:
from tempor import plugin_loader
from data_loaders.temporAI_dataloader import load_gsu_dataset
import numpy as np

In [4]:
data_path = '/Users/jk1/temp/treatment_effects/preprocessing/gsu_Extraction_20220815_prepro_25112023_213851/preprocessed_features_25112023_213851.csv'
continuous_outcomes_path = '/Users/jk1/temp/treatment_effects/preprocessing/gsu_Extraction_20220815_prepro_25112023_213851/preprocessed_outcomes_continuous_25112023_213851.csv'

# Load GSU dataset

In [5]:
gsu_dataset = load_gsu_dataset(data_path, continuous_outcomes_path)

In [6]:
gsu_dataset

TemporalTreatmentEffectsDataset(
    time_series=TimeSeriesSamples([2493, *, 84]),
    predictive=TemporalTreatmentEffectsTaskData(
        targets=TimeSeriesSamples([2493, *, 1]),
        treatments=TimeSeriesSamples([2493, *, 1])
    )
)

## Explore model

In [7]:
temp_dataset = gsu_dataset[0:2]

In [9]:
temp_dataset.predictive.treatments.dataframe().anti_hypertensive_strategy.unique()

array([4., 0., 7.])

In [10]:
params = {
                                  'epochs': 2,
                                  'encoder_rnn_type': 'LSTM',
                              'encoder_hidden_size': 3,
                              'encoder_num_layers': 2,
                              'encoder_bias': True,
                              'encoder_dropout': 0.0,
                              'encoder_bidirectional': False,
                              'encoder_nonlinearity': None,
                              'encoder_proj_size': None,
                              'decoder_rnn_type': 'LSTM',
                              'decoder_hidden_size': 100,
                              'decoder_num_layers': 1,
                              'decoder_bias': True,
                              'decoder_dropout': 0.0,
                              'decoder_bidirectional': False,
                              'decoder_nonlinearity': None,
                              'decoder_proj_size': None,
                              'adapter_hidden_dims': [50],
                              'adapter_out_activation': 'Tanh',
                              'predictor_hidden_dims': [],
                              'predictor_out_activation': None,
                              'max_len': None,
                              'optimizer_str': 'Adam',
                              'optimizer_kwargs': {'lr': 0.01, 'weight_decay': 1e-05},
                              'batch_size': 32,
                              'padding_indicator': -999.0
}

In [11]:
# Initialize the model:
model = plugin_loader.get("treatments.temporal.regression.crn_regressor", epochs=params['epochs'], encoder_rnn_type=params['encoder_rnn_type'], encoder_hidden_size=params['encoder_hidden_size'], encoder_num_layers=params['encoder_num_layers'], encoder_bias=params['encoder_bias'], encoder_dropout=params['encoder_dropout'], encoder_bidirectional=params['encoder_bidirectional'], encoder_nonlinearity=params['encoder_nonlinearity'], encoder_proj_size=params['encoder_proj_size'], decoder_rnn_type=params['decoder_rnn_type'], decoder_hidden_size=params['decoder_hidden_size'], decoder_num_layers=params['decoder_num_layers'], decoder_bias=params['decoder_bias'], decoder_dropout=params['decoder_dropout'], decoder_bidirectional=params['decoder_bidirectional'], decoder_nonlinearity=params['decoder_nonlinearity'], decoder_proj_size=params['decoder_proj_size'], adapter_hidden_dims=params['adapter_hidden_dims'], adapter_out_activation=params['adapter_out_activation'], predictor_hidden_dims=params['predictor_hidden_dims'], predictor_out_activation=params['predictor_out_activation'], max_len=params['max_len'], optimizer_str=params['optimizer_str'], optimizer_kwargs=params['optimizer_kwargs'], batch_size=params['batch_size'], padding_indicator=params['padding_indicator'],
                          encoder_fc_hidden_units=48
)

In [12]:
# Train the model:
model.fit(temp_dataset)

Preparing data for decoder training...
Preparing data for decoder training DONE.
=== Training stage: 1. Train encoder ===
Epoch: 0, Prediction Loss: 3.070, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 3.070
Epoch: 1, Prediction Loss: 2.805, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 2.805
=== Training stage: 2. Train decoder ===
Epoch: 0, Prediction Loss: 0.066, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 0.066
Epoch: 1, Prediction Loss: 0.022, Lambda: 1.000, Treatment BR Loss: 0.000, Loss: 0.022


CRNTreatmentsRegressor(
    name='crn_regressor',
    category='treatments.temporal.regression',
    plugin_type='method',
    params={
        'encoder_rnn_type': 'LSTM',
        'encoder_hidden_size': 3,
        'encoder_num_layers': 2,
        'encoder_bias': True,
        'encoder_dropout': 0.0,
        'encoder_bidirectional': False,
        'encoder_nonlinearity': None,
        'encoder_proj_size': None,
        'decoder_rnn_type': 'LSTM',
        'decoder_hidden_size': 100,
        'decoder_num_layers': 1,
        'decoder_bias': True,
        'decoder_dropout': 0.0,
        'decoder_bidirectional': False,
        'decoder_nonlinearity': None,
        'decoder_proj_size': None,
        'adapter_hidden_dims': [50],
        'adapter_out_activation': 'Tanh',
        'predictor_hidden_dims': [],
        'predictor_out_activation': None,
        'max_len': None,
        'optimizer_str': 'Adam',
        'optimizer_kwargs': {'lr': 0.01, 'weight_decay': 1e-05},
        'batch_size': 32,

In [14]:
# Define horizons for each sample.
horizons = [tc.time_indexes()[0][len(tc.time_indexes()[0]) // 2 :] for tc in temp_dataset.time_series]
print("Horizons for sample 0:\n", horizons[0], end="\n\n")

Horizons for sample 0:
 [35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]


In [46]:
# Define treatment scenarios for each sample.
treatment_scenarios = [[np.asarray([2] * len(h)), np.asarray([0] * len(h))] for h in horizons]
print("Alternative treatment scenarios for sample 0:\n", treatment_scenarios[0], end="\n\n")

Alternative treatment scenarios for sample 0:
 [array([0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])]


In [47]:
# Actual treatment scenarios for sample 0:
temp_dataset.predictive.treatments[0]

TimeSeriesSamples with data:
                      anti_hypertensive_strategy
sample_idx  time_idx                            
100023_4784 0                                4.0
            1                                4.0
            2                                4.0
            3                                4.0
            4                                4.0
...                                          ...
            66                               0.0
            67                               0.0
            68                               0.0
            69                               0.0
            70                               0.0

[71 rows x 1 columns]

In [48]:
# Call predict_counterfactuals.
counterfactuals = model.predict_counterfactuals(temp_dataset, horizons=horizons, treatment_scenarios=treatment_scenarios)

AssertionError: 

In [18]:
print("Counterfactual outcomes for sample 0, given the alternative treatment scenarios:\n")
for idx, c in enumerate(counterfactuals[0]):
    print(f"Treatment scenario {idx}, {treatment_scenarios[0][idx]}")
    print(c, end="\n\n")

Counterfactual outcomes for sample 0, given the alternative treatment scenarios:

Treatment scenario 0, [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
TimeSeries() with data:
          nihss_delta_at_next_ts
time_idx                        
35                      0.073940
36                      0.045756
37                      0.046453
38                      0.046436
39                      0.046436
40                      0.046436
41                      0.046436
42                      0.046436
43                      0.046436
44                      0.046436
45                      0.046436
46                      0.046436
47                      0.046436
48                      0.046436
49                      0.046436
50                      0.046436
51                      0.046436
52                      0.046436
53                      0.046436
54                      0.046436
55                      0.046436
56                      0.046436
57    

In [13]:
# import os
# from tempor.utils.serialization import save_to_file
# 
# save_to_file(os.path.join('/Users/jk1/temp/treatment_effects/training_test', f'crn_model_{101}_split_{0}'), model)
